<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### TP 2
### - Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec


import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

    df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)

from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

from gensim.models.callbacks import CallbackAny2Vec
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=1,
                     sg=1)

w2v_model.build_vocab(sentence_tokens)

w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True
                 )


El dataset ya se encuentra descargado


/tmp/ipython-input-4273192447.py:21: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



(218093, 359260)

### - Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

In [47]:
print("Palabras que mas se relacionan con 'baby' \n")
w2v_model.wv.most_similar(positive=["no"], topn=10)

Palabras que mas se relacionan con 'baby' 



[('tears', 0.9265539646148682),
 ('woman', 0.9244767427444458),
 ('fowl', 0.9166396260261536),
 ('cry', 0.9063509702682495),
 ('call', 0.882744550704956),
 ('shed', 0.8665948510169983),
 ('philosophy', 0.861731231212616),
 ('matter', 0.8615086078643799),
 ('nice', 0.8439921736717224),
 ("ain't", 0.8383564949035645)]

In [48]:
print("Palabras que mas se relacionan con 'woman' \n")
w2v_model.wv.most_similar(positive=["woman"], topn=10)


Palabras que mas se relacionan con 'woman' 



[('cry', 0.9774102568626404),
 ('fowl', 0.9347646832466125),
 ('no', 0.9244766235351562),
 ('tears', 0.916095495223999),
 ('call', 0.8935620784759521),
 ('matter', 0.876559317111969),
 ("ain't", 0.8528661727905273),
 ('sunshine', 0.8423731923103333),
 ('shed', 0.8393506407737732),
 ('nice', 0.8340551853179932)]

In [49]:
print("Palabras que menos se relacionan con 'love' \n")
w2v_model.wv.most_similar(negative=["paradise"], topn=10)

Palabras que menos se relacionan con 'love' 



[('nobody', -0.16903094947338104),
 ('baby', -0.1922057718038559),
 ('love', -0.2094326764345169),
 ('rights', -0.21247272193431854),
 ("can't", -0.22369955480098724),
 ('give', -0.23054803907871246),
 ('up', -0.23286142945289612),
 ('for', -0.23383262753486633),
 ('your', -0.24442517757415771),
 ('come', -0.24734511971473694)]

In [46]:
print("Palabras que menos se relacionan con 'holy' \n")
w2v_model.wv.most_similar(negative=["cry"], topn=10)


Palabras que menos se relacionan con 'holy' 



[("jammin'", -0.21211275458335876),
 ('music', -0.21300330758094788),
 ('too', -0.22456522285938263),
 ('think', -0.2274266630411148),
 ('in', -0.22967185080051422),
 ('jam', -0.23214535415172577),
 ('jamaica', -0.23959660530090332),
 ("we're", -0.24005140364170074),
 ('youths', -0.24022968113422394),
 ('world', -0.2416675090789795)]

Los embeddings muestran que algunas palabras se relacionan conceptualmente (ejemplo: "love" se opone semánticamente a "hypocrites"). Otras asociaciones como "baby" y "woman" surgen simplemente porque Bob Marley las usa frecuentemente juntas en las mismas líneas de sus canciones. El modelo captura tanto relaciones de significado como patrones de co-ocurrencia textual específicos del vocabulario.

### 5 - Visualizar agrupación de vectores

In [57]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels


vecs, labels = reduce_dimensions(w2v_model,3)

MAX_WORDS=200
fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab")

In [58]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### Como conclusión, en la representación tSNE las palabras se agrupan principalmente por co-ocurrencia en las canciones, como "movement", "of", "jah", "people" o "dont", "worry", "about", "thing" que aparecen juntas en los títulos. Esto probablemente se debe al corpus pequeño de 2218 versos, que hace que los embeddings aprendan más patrones de frases repetidas que relaciones semánticas reales. Con más datos, el modelo capturaría mejor las asociaciones conceptuales en lugar de memorizar fragmentos de canciones.